# Comparison of minimization routines in Cython and Matlab

This serves as a testing platform to learn how to implement the GSL minimization routine in Cython. As Economists tend to use Matlab as a default platform this will form the benchmark. 

The exercise through which we will learn how to use the GSL algorithms implements through a Monte-Carlo simulation the asymptotic convergence of the exponential distribution. This will be a one-dimensional minimization routine. 

A single observation drawn from the exponential distribtuion has the following pdf when $\lambda>0$:

\begin{equation*}
f_{Y}(y;p_{0}) = \lambda\cdot e^{-\lambda\cdot y}
\end{equation*}

We will suppose we have an i.i.d. sample of observations of y, so we can take the joint pdf of the y samples as:

\begin{align*}
f_{Y}(y_{1},...,y_{n};p_{0}) &= \prod_{i=1}^{n}\lambda\cdot e^{-\lambda\cdot y_{i}}\\
&=\lambda^{n}\cdot e^{-\lambda\sum_{i=1}^{n}y_{i}}
\end{align*}

Applying logs to the above equation gives us the log-likelihood function:

\begin{equation*}
\mathcal{L}(p_{0};y_1...y_n) = \log f\left(y_1,...,y_n; p_{0}\right)=n\log(\lambda) - \lambda\sum_{i=1}^{n}y_{i}
\end{equation*}

We will set up a Monte-Carlo by using the population lambda as $\lambda=0.67$ to generate a vector y, that we will then use as our observed variable. For each sample size we will run 1000 simulations.

The maximum likelihood estimator of $\widehat{\lambda}$ is $\frac{1}{\bar{y}}$, where $\bar{y}$ is the average of the observed values. Despite this we will maximize the likelihood function to test the speed comparisons and learn to use GSL in Cython. Since we will have to run many minimization routines on different sample sizes we will write an external function that estimates the probability through maximization. We know that $\lambda>0$. We will make use of this as Matlab2015b allows for parallelization of fmincon, but not fminunc.

 ## Matlab
 
 Load matlab magic and set up parallel (optionally remove parallel implementation for a setial comparison).

In [1]:
%load_ext pymatbridge

/usr/local/lib/python3.6/site-packages/IPython/nbformat.py:13: ShimWarning: The `IPython.nbformat` package has been deprecated. You should import from nbformat instead.
  "You should import from nbformat instead.", ShimWarning)


Starting MATLAB on ZMQ socket ipc:///tmp/pymatbridge-60a376da-06e4-49aa-aff9-797d256b129a
Send 'exit' command to kill the server
.......MATLAB started and connected!


In [2]:
%%matlab
clear all; clc; parpool

Starting parallel pool (parpool) using the 'local' profile ... connected to 4 workers.

ans = 

 Pool with properties: 

            Connected: true
           NumWorkers: 4
              Cluster: local
        AttachedFiles: {}
          IdleTimeout: 30 minute(s) (30 minutes remaining)
          SpmdEnabled: true



In [3]:
%%matlab

%We are asked for sample sizes N=5,50,500. Let's create a
%vector of these samples sizes (I did this with an extra
%sample size to get nicer graph outputs).

N=[5;50;500;5000];

%population probability
lam_true = 0.67;

%Simulation run
T=1000;

%Initialize the vectors for the output of the Monte Carlo
%We will initialize a vector of 1000x4, since we have 4
%sample sizes, that is the size of the N vector we created
%is 4x1.
lam_hat = NaN(T,size(N,1));

%A starting value of guess for the p_hat that we will find
lam_0 = rand;

%Our options for the maximization routine:

options = optimoptions('fmincon','Algorithm','interior-point',...
                         'Display','off','UseParallel',true);

In [4]:
%%matlab

%Program the Monte-Carlo experiment

%outer loop runs the sample sizes

tic
for i=1:size(N,1)

    sample_size = N(i);
    
    %run inner loop = MC loop - to make this go fast 
    %we will need to parallelize this otherwise it will
    %take forever
    
    parfor j=1:T
    
        %First we generate a random sample of points
        %between 0 and 1. We use the rand function 
        %to do this
        
        Y = exprnd(lam_true,sample_size,1);
        
        %Now we can run the fmincon routine on the sample Y 
        %and find the lambda that maximizes the likelihood of 
        %observing our sample vector Y:
        
        lam_hat(j,i) = fmincon(@(lam) Exponential(lam,sample_size,Y),...
                               lam_0,[],[],[],[],0,[],[],options)
    
    end

end
toc

Elapsed time is 17.593803 seconds.


In [5]:
%%matlab

lam_mean = mean(lam_hat)


lam_mean =

    1.8598    1.5290    1.4953    1.4925



In [6]:
%%matlab
delete(gcp('nocreate'))

Parallel pool using the 'local' profile is shutting down.


## Cython/Python

Here we will program the above cells in Python and call a Monte-Carlo function to yield us the outputs we desire. The Monte-Carlo function will be written in Cython and we will use the %timeit cell magic to give us a run-time of our Cython function. 

### Cython Function

In [7]:
%load_ext cython

In [14]:
%%cython -lgsl -lgslcblas

#!python
#cython: boundscheck=False, wraparound=False, nonecheck=False, cdivision=True

#Import modules: 

from libc.stdlib cimport rand, RAND_MAX, malloc, calloc, realloc, free, abort
from libc.math cimport log

#Use the CythonGSL package to get the low-level routines
from cython_gsl cimport *

######################### Define the Data Structure ############################

cdef struct Parameters:
    #Pointer for Y data array
    double* Y
    #size of the array
    int* Size

################ Support Functions for Monte-Carlo Function ##################

#Create a function that allocates the memory and verifies integrity
cdef int alloc_struct(Parameters* data, int* N, unsigned int flag) nogil:
    
    cdef int Mem_Int = True
    
    #fill in the size of the array
    data.Size = N
    
    #allocate the data array initially
    if flag==0:
        data.Y = <double*> calloc(N[0], sizeof(double))
    #reallocate the data array according to the size of N
    else:
        data.Y = <double*> realloc(data.Y, N[0] * sizeof(double))
    
    #If the elements of the struct are not properly allocated, destory it and return null
    if N[0]!=0 and data.Y==NULL:
        
        #return the memory to system
        destroy_struct(data)
        
        #update the memory integrity variable to False
        Mem_Int = False
    
    return Mem_Int

#Create the destructor of the struct to return memory to system
cdef void destroy_struct(Parameters* data) nogil:
    free(data.Y)
    free(data)

#This function fills in the Y observed variable with discreet 0/1
cdef void Y_fill(Parameters* data, gsl_rng* r, double lam) nogil:
    
    cdef:
        Py_ssize_t i
        double y
        
    for i in range(data.Size[0]):
        
        data.Y[i] = gsl_ran_exponential(r, lam)

#Definition of the function to be maximized: LLF of the Exponential
cdef double LLF(double lam, void* data) nogil:
    
    cdef:
        #the sample structure (considered the parameter here)
        Parameters* sample = <Parameters*> data
        
        #the loop iterator
        Py_ssize_t i 
        int n = sample.Size[0]
        
        #the total of the LLF
        double Sum = n*log(lam)
     
    for i in range(n):
        
        Sum -= lam*sample.Y[i]
    
    return (-(Sum/n))

########################## Monte-Carlo Function ##############################

cpdef void Monte_Carlo(int[::1] Samples, double[:,::1] lam_hat, double lam_true, int Sims) nogil:
     
    #Define variables and pointers
    cdef:
        #Data Structure
        Parameters* Data
            
        #iterators
        Py_ssize_t i, j
        int status, GSL_CONTINUE, max_Iter = 10000, Iter
        
        #Variables
        int N = Samples.shape[0], Mem_Int
        double a, b, tol = 1e-6, start_val
        
        #define the GSL RNG variables
        const gsl_rng_type* T 
        gsl_rng* r 
        
        #GSL Minimization Objects
        const gsl_min_fminimizer_type* U
        gsl_min_fminimizer* s
        gsl_function F
        
    #allocate the struct dynamically
    Data = <Parameters*> malloc(sizeof(Parameters))
    
    #Allocate the minimization routine
    U = gsl_min_fminimizer_brent
    s = gsl_min_fminimizer_alloc(U)
    
    #Instantiate the RNG
    gsl_rng_env_setup()
    
    T = gsl_rng_default
    r = gsl_rng_alloc(T)
    
    #Verify memory integrity of allocated objects
    if Data==NULL or s==NULL or r==NULL: abort()
    
    #Set the GSL function
    F.function = &LLF
    F.params = <void*> Data
    
    try:
        for i in range(N): 

            #allocate the elements of the struct (if i>0, reallocate)
            Mem_Int = alloc_struct(Data, &Samples[i], i)

            #verify memory integrity of the allocated Struct
            if Mem_Int==False: abort() 
                
            for j in range(Sims):

                #Randomly set the seed
                gsl_rng_set(r, rand())

                #fill the array in the struct
                Y_fill(Data, r, lam_true)

                #set the parameters in GSL F Function
                a = tol; b = 1000

                #set the starting value (random number)
                start_val = rand()/<double>RAND_MAX

                #set the minimizer
                gsl_min_fminimizer_set(s, &F, start_val, a, b)

                #initialize conditions
                GSL_CONTINUE = -2
                status = -2
                Iter = 0

                #maximize the function
                while (status == GSL_CONTINUE and Iter < max_Iter):

                    Iter += 1
                    status = gsl_min_fminimizer_iterate(s)

                    start_val = gsl_min_fminimizer_x_minimum(s)
                    a = gsl_min_fminimizer_x_lower(s)
                    b = gsl_min_fminimizer_x_upper(s)

                    status = gsl_min_test_interval(a, b, tol, 0.0)

                    if (status == GSL_SUCCESS):
                        lam_hat[i,j] = start_val

    finally:
        destroy_struct(Data)
        gsl_rng_free(r)
        gsl_min_fminimizer_free(s)

### Python Portion (call the Cythonized Monte-Carlo function)

In [9]:
import numpy as np

In [10]:
#First we will recreate the first cell in the matlab example in python

#Sample Sizes
N = np.array([5,50,500,5000], dtype='i')

#Parameters for MC
T = 1000
lam_true = 0.67

#Array of the outputs from the MC
lam_hat = np.empty((N.size,T), dtype='d')
lam_hat.fill(np.nan)

In [11]:
%timeit Monte_Carlo(N, lam_hat, lam_true, T)

1 loop, best of 3: 254 ms per loop


In [12]:
print(lam_hat.mean(axis=1))

[ 1.89981833  1.52253325  1.49448779  1.49254098]


In [13]:
print('The true lamba is {}'.format(1/lam_true))

The true lamba is 1.4925373134328357


## Results

When Matlab is parallelized the program takes 17.027 seconds to run (we MUST use it in parallel or this will take an impossibly long time - for example, if you use fminunc in Matlab2015b, which doesn't support parallelization, it takes too long to complete the MC experiment as it has to run through 4000 simulations).

However, when we use the Cython function we complete the minimization routines in 254 milliseconds (0.254 seconds) on average. The code is slightly more complicated but it isn't impossible. Note that using pure python would likely results in speeds compoarable to Matlab, and we would likely have to make use of multiprocessing as well to make the computation feasible. Cython code *might* be faster than Matlab and Pure Python code using the minimization routines in the SciPy stack; but it will definitely be slower than this code as there will be overhead costs of making calls to the Python C-API. 

Using Cython and GSL results in a code that runs **67** times faster than in Matlab. 